Run the following cells to prepare your working environment:

In [ ]:
# !pip install -r requirements.txt

In [ ]:
%load_ext autoreload
%autoreload 2

## Part 1: Using Hierarchical Probabilistic U-Net for Source Reconstruction

In [ ]:
print("hello")

hello


In [ ]:
tuple(range(1,2))

In [ ]:
tuple(range(1,3))

(1, 2)